In [1]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime      
import yfinance as yf
import matplotlib.pyplot
import seaborn as sns
import numpy as np

In [10]:
start = datetime(2020, 1, 1)
end = datetime(2023, 1, 1)

### Análise Superficial do Retorno Cumulativo Percentual nos ativos da B3

In [20]:
def load_ticker(ticker, start, end):
    df = yf.download(ticker, start, end)
    return df

def close_serie(df):
    dates = df.index
    close = df['Close']
    data = pd.DataFrame(close, dates)
    return data, dates

def load_return(df, dates):
    close = df['Close']
    anterior = close.iloc[0]
    pr_list = []
    lr_list = []

    cumulative_return_percentual = 1
    crp_hist = []

    cumulative_log_return = 1
    clr_hist = []
    for close_value in close:
        percentual_return = (close_value/anterior) - 1
        cumulative_return_percentual *= (1+percentual_return)

        log_return = np.log(close_value) - np.log(anterior)
        cumulative_log_return *= (1+log_return)

        pr_list.append(percentual_return)
        lr_list.append(log_return)
        crp_hist.append(cumulative_return_percentual)
        clr_hist.append(cumulative_log_return)

        anterior = close_value

    df_return = pd.DataFrame({'Percentual Return': pr_list}, index=dates)
    df_return['Logaritmal Return'] = lr_list
    df_return['Cumulative Percentual Return'] = crp_hist
    df_return['Cumulative Log Return'] = clr_hist

    return df_return, cumulative_return_percentual

def load_risk(df, dates):
    # std = np.std(np.array(df['Close']))
    span = 3
    std_df = pd.DataFrame(df['Close'], index=dates)

    std_df['Standart Deviation'] = std_df['Close'].rolling(window=3).std()
    std_df['EWMA'] = std_df['Close'].ewm(span=span).std()
    return std_df

In [44]:
tickers = ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'BBDC4.SA', 'ABEV3.SA', 'MGLU3.SA']
carteira = pd.DataFrame(columns=['ticker', 'percentual return'])

for ticker in tickers:
    df = load_ticker(ticker, start, end)
    close_array, dates = close_serie(df)
    return_df, return_percentual = load_return(df,dates)
    risk_df = load_risk(df, dates)
    carteira.loc[len(carteira)] = [ticker, return_percentual]

weight = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]
carteira['peso'] = weight
carteira['weighted return'] = np.array(weight*np.array(carteira['percentual return']))
capital = 1000
lucro = np.sum(np.array(carteira['weighted return'])*capital) - capital
if lucro > 0:
    print('com o capital investido de R$', capital, 'com a carteira igualmente distribuida entre os ativos ',tickers,', o lucro calculado entre 2020/01/01 a 2023/01/01 é de R$', lucro)
else:
    print('com o capital investido de R$', capital, 'com a carteira igualmente distribuida entre os ativos ',tickers,', a perda calculada entre 2020/01/01 a 2023/01/01 é de R$', np.abs(lucro))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

com o capital investido de R$ 1000 com a carteira igualmente distribuida entre os ativos  ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'BBDC4.SA', 'ABEV3.SA', 'MGLU3.SA'] , a perda calculada entre 2020/01/01 a 2023/01/01 é de R$ 232.10705913933532
